# 딥러닝스니펫

#### - 모델링 전
1. 모듈 임포트
2. 데이터 불러오기
3. 학습, 평가 데이터 분리
4. reshape
5. scaling
6. one-hot encoding

#### - Sequential API, Functional API
1. 세션 클리어
2. 모델 선언
3. 레이어 조립
4. 컴파일
5. 요약

#### - EarlyStopping

#### - 시각화

# 모듈 가져오기

#### 기본

In [ ]:
import numpy as np
import pandas as pd

#### 딥러닝

In [ ]:
import tensorflow
from tensorflow import keras

from keras.utils import to_categorical, plot_model
from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

#### 시각화

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('font', family='Malgun Gothic')
sns.set(font="Malgun Gothic",
        rc={"axes.unicode_minus":False}, # 마이너스 부호 깨짐 현상 해결
        style='darkgrid')  

import warnings
warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format = 'retina'

# 모델링 전

1. 모듈 임포트
2. 데이터 불러오기
3. 학습, 검증, 평가 데이터 분리
4. reshape
5. scaling
6. one-hot encoding

#### x, y 분리

In [ ]:
target='타겟컬럼'

x = df.drop(target, axis=1)
y = df.loc[:, target]

#### train, val, test 데이터 분리

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.1, random_state=2023)
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.1, random_state=2023)

In [ ]:
train_x.shape, train_y.shape, val_x.shape, val_y.shape

#### reshape

In [ ]:
train_x = train_x.reshape( (train_x.shape[0], -1) )
test_x = test_x.reshape( (test_x.shape[0], -1) )

#### min-max scaling

In [ ]:
max_n, min_n = train_x.max(), train_x.min()

train_x = (train_x - min_n) / (max_n - min_n)
test_x = (test_x - min_n) / (max_n - min_n)

#### one-hot encoding

In [ ]:
class_n = len(set(train_y))

train_y = to_categorical(train_y, class_n)
test_y = to_categorical(test_y, class_n)

# Sequential API

In [ ]:
# 1번. 세션 클리어 : 기존의 모델 구조가 남아있다면 지워줘
keras.backend.clear_session()

# 2번. 모델 선언 : 레이어 블록을 조립할 발판!
model = keras.models.Sequential()

# 3번. 레이어 조립 : .add()
model.add(keras.layers.Input(shape=(784,)) )
model.add(keras.layers.Dense(10, activation='relu') )
model.add(keras.layers.Dense(10, activation='swish') )
model.add(keras.layers.Dense(10, activation='sigmoid') )
model.add(keras.layers.Dense(3, activation='softmax') )

# 4번. 컴파일
model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
              optimizer='adam')

# 요약
model.summary()

# Functional API

In [ ]:
# 1번 세션 클리어
keras.backend.clear_session()

# 2번 레이어 엮기
il1 = keras.layers.Input(shape=(13,))
hl1 = keras.layers.Dense(64, activation='relu', name='hidden1')(il1)

il2 = keras.layers.Input(shape=(13,))
hl2 = keras.layers.Dense(64, activation='relu', name='hidden2')(il2)

add_l = keras.layers.Add()([hl1, hl2])
concat_l = keras.layers.Concatenate()([hl1, hl2])
ol = keras.layers.Dense(3, activation='softmax', name='output')(add_l) # concat_l

# 3번 모델의 시작/끝 지정
model = keras.models.Model([il1, il2], ol)

# 4번 컴파일
model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
              optimizer='adam')

# 요약
model.summary()

In [ ]:
plot_model( model , show_shapes=True, show_layer_names=True )

# EarlyStopping

In [ ]:
es = EarlyStopping(
                    monitor='val_loss',         # 관측 대상
                    min_delta=0,                # 임계값. 관측 대상이 임계값만큼 변하지 않는다면 patience 1 증가
                    patience=5,                 # 관측 대상 성능이 개선되지 않을 때 몇 번 더 지켜볼래
                    verbose=1,                  # 학습과정 보이기
                    restore_best_weights=True   # 최적의 epoch 가중치를 모델에 적용
                  )  

In [ ]:
history = model.fit([il1, il2], train_y, validation_split=0.2, epochs=10000, verbose=1,
          callbacks=[es])

In [ ]:
y_pred = model.predict([test_x_1, test_x_2])

# 시각화

In [ ]:
"""
train, val data 의 loss, accuracy 시각화
- history
- epoch
"""

def plot_loss_acc( history, epochs ) :
    loss, val_loss = history.history["loss"], history.history["val_loss"]
    acc, val_acc = history.history["accuracy"], history.history["val_accuracy"]

    fig, ax = plt.subplots(1,2, figsize=(12,4))

    ax[0].plot(range(1, epochs+1), loss, label="Training")
    ax[0].plot(range(1, epochs+1), val_loss, label="Validation")
    ax[0].legend(loc="best")
    ax[0].set_title("Loss")

    ax[1].plot(range(1, epochs+1), acc, label="Training")
    ax[1].plot(range(1, epochs+1), val_acc, label="Validation")
    ax[1].legend(loc="best")
    ax[1].set_title("Accuracy")

    plt.show()

In [ ]:
import matplotlib.pyplot as plt
"""
epochs: 에폭횟수!
"""
epochs = 
plot_loss_acc(history, epochs)

# 저장 및 로드

In [ ]:
model.save('model.h5')

In [ ]:
model = keras.models.load_model('path/to/location')